In [29]:
import pandas as pd 

def load_suspend_data(datasets, imu, csv_file):
    datasets += ['GT']
    csv_file = 'Suspend/'+csv_file
    df_dict = load_csv_files(datasets, imu, csv_file)
    return df_dict

def load_frequency_data(datasets, imu, csv_file, freq_map):
    datasets += ['AC']
    csv_file = 'Frequency/'+csv_file
    df_dict = load_csv_files(datasets, imu, csv_file)

    df = df_dict['AC']
    # load frequency map for frequency attacks
    df2 = df.replace({"Ground_Truth":freq_map})
    df['Ground_Truth'] = df2['Ground_Truth']
    return df_dict

def load_csv_files(datasets, imu, csv_file):
    df_dict = dict()
    csv_dir = 'logs/'+imu+'/'
    
    for ds in datasets:
        print('loading: ',csv_dir+csv_file+ds+'.csv')
        df = pd.read_csv(csv_dir+csv_file+ds+'.csv')
        df_dict[ds]= df
    return df_dict
    

def compute_transition_df(source_df, param):
    # dataframe for attack transition
    df_transition = pd.DataFrame()
    df_transition['timestamp'] = source_df['timestamp']
    df_transition[param] = source_df[param]
    df_transition['transition'] = source_df[param].diff()
    df_transition = df_transition.loc[df_transition['transition']!=0].dropna()
    return df_transition

# Processing CSV files

In [30]:
import pandas as pd
import plotly.express as px
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# File names
csv_dir = 'logs/'
csv_file1 = 'MultipleFrequenciesAttack_'
csv_file2 = 'IMU_at_true_speed_'            # (BMI270) Frequency attack on our board with logging at sensor speed.
csv_file3 = 'Pixhawk_multiple_freq_'
csv_file4 = 'PixHawk6C_RT_multipleFreq_'    # (BMI055) Frequency attack on PixHawk6C board with logging at sensor speed.
csv_file5 = 'suspend_bmi055_v7_'            # (BMI055) Suspend attack
csv_file6 = 'Invensensev3Fequency_'         # (ICM42688)
csv_file7 = 'Invensensev3Suspend_'          # (ICM42688)
csv_file2 = 'suspend_with_true_speed_IMU_'
csv_file2 = 'suspend_'

# kakute_board = True
frequency_attack = False
IMU_list = ['BMI270', 'BMI055', 'ICM42688', 'MPU6000']
experiment_data = {'BMI270': 
                    {'suspend_file': 'suspend_',
                     'freq_file': 'frequency_all_possible_',
                     'freq_map':{0:3200, 1:1600, 2:800, 3:400, 4:200, 5:100, 6:50, 7:25, 8:25/2, 9:25/4, 10:25/8, 11:25/16 ,12:25/32}},
                    'BMI055':
                    {'suspend_file': csv_file5,},
                    'MPU6000':
                    {'suspend_file': 'InvensenseSuspend_',}
                    }



IMU = IMU_list[0]
csv_dir += str(IMU) + '/'

freq_map_file2 = {0:1600, 1:800, 2:400, 3:200, 4:100, 5:50, 6:25}
freq_map_file4 = {15:1000, 14:500, 13:250, 12:125, 11:62.5, 10:31.25, 9:15.63}
freq_map_MPU6000 = {0:256, 1:188, 2:98, 3:42, 4:20, 5:10, 6:5}

# if kakute_board:
#     csv_file = csv_file2
#     freq_map = freq_map_file2
#     # timestamp_limit = 2000
# else:
#     csv_file = csv_file4
#     freq_map = freq_map_file4
#     # timestamp_limit = 75

datasets = ['ATT', 'IMU']

# Adding fast sensor readings 
datasets += ['RGY', 'RAC']

if frequency_attack:
    datasets += ['AC']
    csv_dir += 'Frequency/'
    csv_file = experiment_data[IMU]['freq_file']
    freq_map = experiment_data[IMU]['freq_map']
else:
    datasets += ['GT']
    csv_dir += 'Suspend/'
    csv_file = experiment_data[IMU]['suspend_file']

# load all csv files into a df_dict dictionary
df_dict = dict()
for ds in datasets:
    print('loading: ',csv_dir+csv_file+ds+'.csv')
    df = pd.read_csv(csv_dir+csv_file+ds+'.csv')
    df_dict[ds]= df#.loc[df['timestamp']<timestamp_limit]

if frequency_attack:
    df = df_dict['AC']
    # load frequency map for frequency attacks
    df2 = df.replace({"Ground_Truth":freq_map})
    df['Ground_Truth'] = df2['Ground_Truth']
    param = 'Ground_Truth'
else:
    param = 'Ground_Truth'

# dataframe for attack transition
# df_transition = pd.DataFrame()
# df_transition['timestamp'] = df['timestamp']
# df_transition[param] = df[param]
# df_transition['transition'] = df[param].diff()
# df_transition = df_transition.loc[df_transition['transition']!=0].dropna()
df_transition = compute_transition_df(source_df=df, param='Ground_Truth')

print("[**] transition dataframe")
print(df_transition)


loading:  logs/BMI270/Suspend/suspend_ATT.csv
loading:  logs/BMI270/Suspend/suspend_IMU.csv
loading:  logs/BMI270/Suspend/suspend_RGY.csv
loading:  logs/BMI270/Suspend/suspend_RAC.csv
loading:  logs/BMI270/Suspend/suspend_GT.csv
[**] transition dataframe
        timestamp  Ground_Truth  transition
9060      11.7663           100    100.0000
11262     13.6487             0   -100.0000
18762     19.0938           100    100.0000
21262     20.9784             0   -100.0000
28762     26.4224           100    100.0000
31262     28.3060             0   -100.0000
38762     33.7493           100    100.0000
41262     35.6326             0   -100.0000
48762     41.0762           100    100.0000
51262     42.8748             0   -100.0000
58762     48.3182           100    100.0000
61262     50.2016             0   -100.0000
68762     55.6451           100    100.0000
71262     57.5288             0   -100.0000
78762     62.9725           100    100.0000
81262     64.8557             0   -100.00

## Enriching Datasets

In [31]:

stats_params = {  
            'IMU': 
                ['GyrX', 'GyrY', 'GyrZ', 'AccX', 'AccY', 'AccZ'],
            'RAC':
                ['ACCx','ACCy','ACCz'],
            'RGY':
                ['GYROx','GYROy','GYROz'],
            'ATT':
                ['Roll', 'Pitch', 'Yaw']
        }


# Add attack column into based on timestamps from the transition dataframe
for k in stats_params.keys():
    attack0 = df[param].iloc[0]
    attack = list()
    this_df = df_dict[k]
    tr0 = 0
    for tr in df_transition['timestamp'].values:
        data_size = len(this_df[(this_df['timestamp']>tr0) & (this_df['timestamp']<=tr)])
        # print('adding {} data points to {} points'.format(data_size, len(attack)))
        attack += [attack0]*data_size
        attack0 = df_transition['Ground_Truth'][df_transition['timestamp']==tr].values[0]
        tr0 = tr
    # Compute last segment
    data_size = len(this_df[this_df['timestamp']>tr])
    # print('adding {} data points to {} points'.format(data_size, len(attack)))
    attack += [attack0]*data_size
    this_df['attack'] = attack
    
    # Add 'update' column with the time diference between data points
    this_df['update'] = this_df['timestamp'].diff()

    df_dict[k] = this_df

update_dict = dict()
for ds in datasets[:-1]:
    df_update = pd.DataFrame()
    df_update['timestamp'] = df_dict[ds]['timestamp']
    df_update['update'] = df_dict[ds]['timestamp'].diff()
    update_dict[ds] = df_update
    

# Statistical Results

In [32]:

stat_dict = dict()
param_set = list()

for k,values in stats_params.items():
    for v in values+['update']:
        this_df = df_dict[k]
        # tr0 = 0
        # compute mean and std deviation for all variables in 'stats_params'
        atk_set = this_df['attack'].unique()
        for atk in atk_set:
            if 'update' in v:
                filtered_list = 1/this_df[v][this_df['attack']==atk]
            else:
                filtered_list = this_df[v][this_df['attack']==atk]

            mean_value = filtered_list.mean()
            std_value = filtered_list.std()
            # print('Attack: {}. {}.{} mean :{:.2f} ({:.2f})'.format(atk, k, v, mean_value, std_value))

            # fill up the stat_dict dictionary by attack
            name_code = '{}.{}'.format(k, v)
            param_set.append(name_code)
            if atk in stat_dict:
                    stat_dict[atk][name_code] = {'mean': mean_value, 'std':std_value}
            else:
                stat_dict[atk] = {name_code:{'mean': mean_value, 'std':std_value}}

# convert stats into a dataframe
df_columns = ['Freq']
for i in list(set(param_set)):
    df_columns+= [i+'.mean', i+'.std']
stat_df = pd.DataFrame(columns=df_columns)
stat_df['Freq'] = atk_set
for freq in stat_dict:
    for param in stat_dict[freq]:
        stat_df.loc[stat_df['Freq']==freq,param+'.mean'] = stat_dict[freq][param]['mean']
        stat_df.loc[stat_df['Freq']==freq,param+'.std'] = stat_dict[freq][param]['std']

pd.options.display.float_format = '{:.4f}'.format
print(stat_df.loc[stat_df['Freq']!=0])
print(stat_df[['RAC.ACCx.mean', 'RAC.ACCx.std',
               'RAC.ACCy.mean', 'RAC.ACCy.std',
               'RAC.ACCz.mean', 'RAC.ACCz.std',
               'RGY.GYROx.mean', 'RGY.GYROx.std',
               'RGY.GYROy.mean', 'RGY.GYROy.std',
               'RGY.GYROz.mean', 'RGY.GYROz.std',]].loc[stat_df['Freq']!=0].to_latex())

   Freq RGY.GYROz.mean RGY.GYROz.std IMU.GyrZ.mean IMU.GyrZ.std IMU.GyrY.mean  \
1   100        31.4876        0.0000       31.3117       2.0229       31.3118   

  IMU.GyrY.std ATT.Pitch.mean ATT.Pitch.std ATT.Roll.mean  ... RAC.ACCx.mean  \
1       2.0229       -16.3377       35.2902       25.8229  ...      141.5260   

  RAC.ACCx.std IMU.AccZ.mean IMU.AccZ.std RGY.GYROy.mean RGY.GYROy.std  \
1       0.0000      140.8189       9.0694        31.4877        0.0000   

  IMU.update.mean IMU.update.std RAC.ACCz.mean RAC.ACCz.std  
1        401.6556        26.7853      141.5260       0.0000  

[1 rows x 39 columns]
\begin{tabular}{lllllllllllll}
\toprule
 & RAC.ACCx.mean & RAC.ACCx.std & RAC.ACCy.mean & RAC.ACCy.std & RAC.ACCz.mean & RAC.ACCz.std & RGY.GYROx.mean & RGY.GYROx.std & RGY.GYROy.mean & RGY.GYROy.std & RGY.GYROz.mean & RGY.GYROz.std \\
\midrule
1 & 141.526047 & 0.000000 & 141.526047 & 0.000000 & 141.526047 & 0.000000 & 31.481480 & 0.000000 & 31.487732 & 0.000000 & 31.487648 & 0

## Plotting Suspend attack results

In [33]:
from plot_formatting import beautify_plot

fig = make_subplots(rows=6, cols=1,
                    shared_xaxes=True,
                    vertical_spacing = 0.02,
                    row_heights=[.05,.25]*2 + [.2]*2)

colorset = px.colors.qualitative.Bold

# Plot data source
plot_params = {  
            # 'IMU': 
            #     ['GyrX', 'AccX'],
            'RAC':
                ['ACCx', 'update'],
            'RGY':
                ['GYROx', 'update'],
            'ATT':
                ['Roll', 'Pitch', 'Yaw', 'update'],
        }


plot_labels = {'RAC':'Accel', 
               'RGY': 'Gyro',
               'ATT': 'EKF Estimation'}

x_range = [84.9,85.1]

color_ix = 0
for k,values in plot_params.items():
    for v in values:
        row_num  = 1
        this_df = df_dict[k]
        this_df = this_df.loc[(this_df['timestamp']>x_range[0]) & (this_df['timestamp']<x_range[1])]
        if 'G' in v:
            row_num = 3
        tr0 = 0
        # Plotting attitude
        if 'ATT' in k and not ('update' in v):
            fig.add_trace(go.Scatter(x=this_df['timestamp'], y=this_df[v],
                    name= v,
                    mode='lines',
                    legendgroup=k,
                    line=dict(color=colorset[color_ix]),
                    showlegend=True,
                    legendrank=3,
                    legendgrouptitle_text='Attitude'),
                row=5,col=1)
        # PLotting update frequencies
        elif 'update' in v:
            fig.add_trace(go.Scatter(x=this_df['timestamp'], y=1/this_df[v],
                    name= plot_labels[k],
                    mode='lines',
                    line=dict(color=colorset[color_ix]),
                    showlegend=True,
                    legendrank=5,
                    legendgroup='freq',
                    legendgrouptitle_text='Frequency'),
                row=6,col=1)
        else:
            fig.add_trace(go.Scatter(x=this_df['timestamp'], y=this_df[v],
                    name= k+str('.')+v,
                    mode='lines',
                    legendgroup=v,
                    line=dict(color=colorset[color_ix]),
                    showlegend=False),
                row=row_num,col=1)
            fig.add_trace(go.Scatter(x=this_df['timestamp'], y=this_df[v],
                    name= v,
                    mode='lines',
                    legendgroup=v,
                    line=dict(color=colorset[color_ix]),
                    showlegend=False),
                row=row_num+1,col=1)


        color_ix += 1


fig2 = go.Figure()


# Plot attacks
param = 'Ground_Truth'
if frequency_attack:
    df = df_dict['AC']
    df = df.loc[(df['timestamp']>x_range[0]) & (df['timestamp']<x_range[1])]
    # load frequency map for frequency attacks
    df2 = df.replace({"Ground_Truth":freq_map})
    df['freq'] = df2['Ground_Truth']
    param = 'freq'
    df_update['freq'] = df['freq']
else:
    df = df_dict['GT']
    df = df.loc[(df['timestamp']>x_range[0]) & (df['timestamp']<x_range[1])]
# fig.add_trace(go.Scatter(x=df['timestamp'], y=df[param],
#                             name='Attack'),
#                     row=6,col=1)

# inlcude attack transitions as vertical lines in the figure 
for v in df_transition['timestamp'][(df_transition['timestamp']>x_range[0]) & (df_transition['timestamp']<x_range[1])].values:
    fig.add_vline(x=v)

# beautify plots
fig = beautify_plot(fig)
fig2 = beautify_plot(fig2)
fig2['layout'].update(plot_bgcolor='#FFFFFF', width=1200, height=600, title='BMI270 - Suspend Attack')

subplot_titles=['value', 'data rate', 'freq'],

#  Formating x-axis
layout = go.Layout( autosize=True, margin={'l': 0, 'r': 0, 't': 0, 'b': 0})
fig['layout'].update(layout)
fig['layout']['xaxis'].update(range=x_range)

fig['layout']['xaxis1'].update(side='top', mirror=False)
fig['layout']['xaxis2'].update(mirror=False)
fig['layout']['xaxis3'].update(side='top', mirror=False)
fig['layout']['xaxis4'].update(mirror=False)
fig['layout']['xaxis6'].update(title='Time [s]', mirror=True)

# Formating y-axis
fig['layout']['yaxis1'].update(range=[141.52,141.53], tickvals = [141.52, 141.53],)
fig['layout']['yaxis2'].update(title='Accel X [m/s/s]')
fig['layout']['yaxis2'].update(range=[0.45,0.55])
fig['layout']['yaxis3'].update(range=[31.47,31.5], tickvals = [31.47,31.5],)
fig['layout']['yaxis4'].update(title='Gyro X [rad/s]')
fig['layout']['yaxis4'].update(range=[-0.015,0.015])
fig['layout']['yaxis6'].update(title='Frequency [Hz]')
fig['layout']['yaxis5'].update(title='Attitude [°]')
fig['layout']['yaxis6'].update(range=[100,1600])

fig['layout']['legend'].update(orientation='h',
                                x=0.45,
                                y=0.6,
                                   )

fig.show()


In [34]:
fig.write_image('figs/BMI270Suspend.pdf')

## Pulling Frequency attack data

In [35]:
import data_processing as dp

IMU_list = ['BMI270', 'BMI055', 'ICM42688', 'MPU6000']

datasets = ['ATT', 'IMU']
# Adding fast sensor readings 
datasets += ['RGY', 'RAC']

df_dict = dp.load_frequency_data(datasets,imu=IMU_list[0])

df_transition = dp.compute_transition_df(source_df=df_dict['AC'], param='Ground_Truth')
print("[**] transition dataframe")
print(df_transition)

[**] transition dataframe
        timestamp  Ground_Truth  transition
770      696.3778      800.0000   -800.0000
10770    703.1534      400.0000   -400.0000
20770    709.6527      200.0000   -200.0000
30770    716.0235      100.0000   -100.0000
40770    722.3335       50.0000    -50.0000
50770    728.6129       25.0000    -25.0000
60770    734.8774       12.5000    -12.5000
70770    741.1352        6.2500     -6.2500
80770    747.4415        3.1250     -3.1250
90770    753.7482        1.5625     -1.5625
100770   760.0550        0.7812     -0.7812
110770   766.3615     3200.0000   3199.2188
120770   773.7731     1600.0000  -1600.0000
130770   781.1014      800.0000   -800.0000
140770   787.8780      400.0000   -400.0000
150770   794.3772      200.0000   -200.0000
160770   800.7483      100.0000   -100.0000
170770   807.0582       50.0000    -50.0000
180770   813.3375       25.0000    -25.0000
190770   819.6018       12.5000    -12.5000
200770   825.8597        6.2500     -6.2500
210770

In [36]:
## Enriching Dataset

df=df_dict['AC']
param='Ground_Truth'
stats_params = {  
            'IMU': 
                ['GyrX', 'GyrY', 'GyrZ', 'AccX', 'AccY', 'AccZ'],
            'RAC':
                ['ACCx','ACCy','ACCz'],
            'RGY':
                ['GYROx','GYROy','GYROz'],
            'ATT':
                ['Roll', 'Pitch', 'Yaw']
        }


# Add attack column into based on timestamps from the transition dataframe
for k in stats_params.keys():
    attack0 = df[param].iloc[0]
    attack = list()
    this_df = df_dict[k]
    tr0 = 0
    for tr in df_transition['timestamp'].values:
        data_size = len(this_df[(this_df['timestamp']>tr0) & (this_df['timestamp']<=tr)])
        # print('adding {} data points to {} points'.format(data_size, len(attack)))
        attack += [attack0]*data_size
        attack0 = df_transition['Ground_Truth'][df_transition['timestamp']==tr].values[0]
        tr0 = tr
    # Compute last segment
    data_size = len(this_df[this_df['timestamp']>tr])
    # print('adding {} data points to {} points'.format(data_size, len(attack)))
    attack += [attack0]*data_size
    this_df['attack'] = attack
    
    # Add 'update' column with the time diference between data points
    this_df['update'] = this_df['timestamp'].diff()

    df_dict[k] = this_df

update_dict = dict()
for ds in datasets[:-1]:
    df_update = pd.DataFrame()
    df_update['timestamp'] = df_dict[ds]['timestamp']
    df_update['update'] = df_dict[ds]['timestamp'].diff()
    update_dict[ds] = df_update



## Statistical Result

stat_dict = dict()
param_set = list()

for k,values in stats_params.items():
    for v in values+['update']:
        this_df = df_dict[k]
        # tr0 = 0
        # compute mean and std deviation for all variables in 'stats_params'
        atk_set = this_df['attack'].unique()
        for atk in atk_set:
            if 'update' in v:
                filtered_list = 1/this_df[v][this_df['attack']==atk]
            else:
                filtered_list = this_df[v][this_df['attack']==atk]

            mean_value = filtered_list.mean()
            std_value = filtered_list.std()

            # fill up the stat_dict dictionary by attack
            name_code = '{}.{}'.format(k, v)
            param_set.append(name_code)
            if atk in stat_dict:
                    stat_dict[atk][name_code] = {'mean': mean_value, 'std':std_value}
            else:
                stat_dict[atk] = {name_code:{'mean': mean_value, 'std':std_value}}

# convert stats into a dataframe
df_columns = ['Freq']
for i in list(set(param_set)):
    df_columns+= [i+'.mean', i+'.std']
stat_df = pd.DataFrame(columns=df_columns)
stat_df['Freq'] = atk_set
for freq in stat_dict:
    for param in stat_dict[freq]:
        stat_df.loc[stat_df['Freq']==freq,param+'.mean'] = stat_dict[freq][param]['mean']
        stat_df.loc[stat_df['Freq']==freq,param+'.std'] = stat_dict[freq][param]['std']

pd.options.display.float_format = '{:.4f}'.format
print(stat_df.loc[stat_df['Freq']!=0])
print(stat_df.to_latex())
    

        Freq RGY.GYROz.mean RGY.GYROz.std IMU.GyrZ.mean IMU.GyrZ.std  \
0  1600.0000         0.0009        0.0030        0.0009       0.0010   
1   800.0000         0.0009        0.0021        0.0009       0.0007   
2   400.0000         0.0009        0.0015        0.0009       0.0005   
3   200.0000         0.0009        0.0011        0.0009       0.0003   
4   100.0000         0.0010        0.0008        0.0010       0.0002   
5    50.0000         0.0009        0.0006        0.0009       0.0002   
6    25.0000         0.0010        0.0005        0.0010       0.0001   
7    12.5000         0.0009        0.0005        0.0009       0.0002   
8     6.2500         0.0009        0.0006        0.0009       0.0002   
9     3.1250         0.0009        0.0005        0.0009       0.0001   
10    1.5625         0.0009        0.0005        0.0009       0.0002   
11    0.7812         0.0009        0.0005        0.0009       0.0002   
12 3200.0000         0.0008        0.0047        0.0008       0.

## Plotting Frequency attack results

In [37]:
import plot_formatting as pfg

layout = go.Layout( autosize=True, margin={'l': 0, 'r': 0, 't': 0, 'b': 0})
fig = go.Figure(layout=layout)
colorset = px.colors.qualitative.Vivid

# Plot data source
plot_params = {  
            # 'IMU': 
            #     ['GyrX', 'AccX'],
            'RAC':
                ['ACCx', 'update'],
            'RGY':
                ['GYROx', 'update'],
            'ATT':
                ['Roll', 'Pitch', 'Yaw', 'update'],
        }

# Legend labels
plot_labels = {'RAC':'Accel', 
               'RGY': 'Gyro',
               'ATT': 'EKF Estimation'}

x_range = [766,850]

color_ix = 0
for k,values in plot_params.items():
    for v in values:
        row_num  = 1
        this_df = df_dict[k]
        this_df = this_df.loc[(this_df['timestamp']>x_range[0]) & (this_df['timestamp']<x_range[1])]
        if 'G' in v:
            row_num = 2

        tr0 = 0
        # if 'ATT' in k and not ('update' in v):
        #     fig.add_trace(go.Scatter(x=this_df['timestamp'], y=this_df[v],
        #             name= v,
        #             mode='lines+markers',
        #             legendgroup=k,
        #             line=dict(color=colorset[color_ix]),
        #             showlegend=True,
        #             legendrank=3,
        #             legendgrouptitle_text='Attitude'),
        #         row=3,col=1)
        # PLotting update frequencies
        if 'update' in v:
            fig.add_trace(go.Scatter(x=this_df['timestamp'], y=1/this_df[v],
                    name= plot_labels[k],
                    mode='lines',
                    line=dict(color=colorset[color_ix]),
                    showlegend=True,
                    # legendrank=5,
                    # legendgroup='freq',
                    # legendgrouptitle_text='Frequency',
                    opacity=0.8))
        # else:
        #     fig.add_trace(go.Scatter(x=this_df['timestamp'], y=this_df[v],
        #             name= v,
        #             mode='lines+markers',
        #             legendgroup=v,
        #             line=dict(color=colorset[color_ix]),
        #             showlegend=False),
        #         row=row_num,col=1)


        color_ix += 1

# Plot attacks
param = 'Ground_Truth'
df = df_dict['AC']

    
df = df.loc[(df['timestamp']>x_range[0]) & (df['timestamp']<x_range[1])]

# inlcude attack transitions as vertical lines in the figure 
for v in df_transition['timestamp'][(df_transition['timestamp']>x_range[0]) & (df_transition['timestamp']<x_range[1])].values:
    fig.add_vline(x=v)


# beautify plots
fig = pfg.beautify_plot(fig)

subplot_titles=['value', 'data rate', 'freq'],

#  Formating x-axis
fig['layout']['xaxis'].update(range=x_range)
fig['layout']['xaxis'].update(title='Time [s]')
fig['layout'].update(width=800, height=400,font_size=25)

# Formating y-axis
# fig['layout']['yaxis1'].update(title='Accelerometer X [m/s/s]')
# fig['layout']['yaxis2'].update(title='Gyro X [rad/s]')
fig['layout']['yaxis'].update(title='Frequency [Hz]')
# fig['layout']['yaxis3'].update(title='Attitude [°]')
fig['layout']['yaxis'].update(range=[0,4], type='log')

fig['layout']['legend'].update(orientation='h',
                                x=0.4,
                                y=1,
                                   )

fig.show()


In [38]:
fig.write_image('figs/BMI270Frequency.pdf')